In [47]:
import pandas as pd
import yfinance as yf
import pickle
from datetime import date
from dateutil.relativedelta import relativedelta
from os.path import exists
import sqlite3
#from sqlalchemy import create_engine

In [67]:
tickers = ['JPM', 'TSLA', 'SPY', 'BRK-B', 'INTC', 'SSO', 'ROM', 'AAPL']

In [68]:
# today's date + 1 for the end date
end_dt = date.today() + relativedelta(days=+1)

# setup sql connection
#engine = create_engine('sqlite:///data/stocks.db', echo=False)
con = sqlite3.connect('data/stocks.db')

# get max dates from stocks.db
max_date = {}
for row in con.execute('SELECT ticker, max(Date) FROM stocks group by ticker ORDER BY ticker'):
    max_date[ row[0] ] = row[1].split()[0]

# loop through each ticker
for sym in tickers:
    
    # setting start date based off the max date in the database
    # always assuming the last entry in the database is a partial day 
    # pull so pulling that again. Otherwise set to 2000
    if sym not in max_date:
        start_dt = '2000-01-01'
    else:
        start_dt = max_date[ sym ]
    
    print( sym + ' ' + str(start_dt) + ' to ' + str(end_dt) )
    
    # download the data we need
    df = yf.download(sym,
              start=start_dt, 
              end=end_dt, 
              progress=False)
    if len( df_prev ) > 0:        
        # merge the dataframes
        df_index = df.index
        df_prev_index = df_prev.index
        mask = ~df_prev_index.isin(df_index)
        #result = pd_prev.loc[mask]
        df = pd.concat([df_prev.loc[mask],df])

    # add the ticker to the df for prep to send to DB
    df['ticker'] = sym

    # saving to temp table. then will delete overlapping data from base table
    df.to_sql('stocks_upload', con=con, if_exists='replace')
    
    # delete from base table where we have over lapping data
    sql = 'delete from stocks where ticker like "' + sym + '" and Date in (select Date from stocks_upload group by Date)'
    con.execute(sql)
    
    # add the newly updated data to the base table
    con.execute("insert into stocks select * from stocks_upload")
    
    # save changes
    con.commit()

# close connection to sqlite
con.close()

JPM 2022-08-04 to 2022-08-06
TSLA 2022-08-04 to 2022-08-06
SPY 2022-08-04 to 2022-08-06
BRK-B 2022-08-04 to 2022-08-06
INTC 2022-08-04 to 2022-08-06
SSO 2022-08-04 to 2022-08-06
ROM 2022-08-04 to 2022-08-06
AAPL 2000-01-01 to 2022-08-06
